In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image

In [3]:
def cuda_var(x):
    x = Variable(x)
    if torch.cuda.is_available():
        return x.cuda()
    return x

In [4]:
class Discriminator(nn.Module):
    """
        Simple Discriminator w/ MLP
    """
    def __init__(self, input_size=784, num_classes=1):
        super(Discriminator, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input_size, 200),
            nn.ReLU(),
            nn.Dropout(),
        )
        self.layer2 = nn.Sequential(
            nn.Linear(200, 200),
            nn.ReLU(),
            nn.Dropout(),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(200, num_classes),
            nn.Sigmoid(),
        )
    
    def forward(self, x):
        y_ = x.view(x.size(0), -1)
        y_ = self.layer1(y_)
        y_ = self.layer2(y_)
        y_ = self.layer3(y_)
        return y_

In [5]:
class Generator(nn.Module):
    """
        Simple Generator w/ MLP
    """
    def __init__(self, input_size=100, num_classes=784):
        super(Generator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_size, 200),
            nn.LeakyReLU(0.2),
            nn.Linear(200, 200),
            nn.LeakyReLU(0.2),
            nn.Linear(200, num_classes),
            nn.Tanh()
        )
        
    def forward(self, x):
        y_ = self.layer(x)
        y_ = y_.view(x.size(0), 1, 28, 28)
        return y_

In [6]:
D = Discriminator().cuda()
G = Generator().cuda()

In [7]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                std=(0.5, 0.5, 0.5))]
)

In [8]:
mnist = datasets.MNIST(root='../data/', train=True, transform=transform, download=True)

In [9]:
batch_size = 64

In [10]:
data_loader = DataLoader(dataset=mnist, batch_size=batch_size, shuffle=True, drop_last=True)

In [11]:
criterion = nn.BCELoss()
D_opt = torch.optim.Adam(D.parameters())
G_opt = torch.optim.Adam(G.parameters())

In [36]:
max_epoch = 10 # need more than 200 epochs for training generator
step = 0
n_critic = 5 # for training more k steps about Discriminator
n_noise = 100

In [37]:
D_labels = cuda_var(torch.ones(batch_size)) # Discriminator Label to real
D_fakes = cuda_var(torch.zeros(batch_size)) # Discriminator Label to fake

for epoch in range(max_epoch):
    for idx, (images, _) in enumerate(data_loader):
        step += 1
        # Training Discriminator
        x = cuda_var(images)
        x_outputs = D(x)
        D_x_loss = criterion(x_outputs, D_labels)

        z = cuda_var(torch.randn(batch_size, n_noise))
        z_outputs = D(G(z))
        D_z_loss = criterion(z_outputs, D_fakes)
        D_loss = D_x_loss + D_z_loss
        
        D.zero_grad()
        D_loss.backward()
        D_opt.step()

        if step % n_critic == 0:
            # Training Generator
            z = cuda_var(torch.randn(batch_size, n_noise))
            z_outputs = D(G(z))
            G_loss = criterion(z_outputs, D_labels)

            D.zero_grad()
            G.zero_grad()
            G_loss.backward()
            G_opt.step()
        
        if step % 1000 == 0:
            print('Epoch: {}/{}, Step: {}, D Loss: {}, G Loss: {}'.format(epoch, max_epoch, step, D_loss.data[0], G_loss.data[0]))        

/home/yangyangii/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1189: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch: 1/10, Step: 1000, D Loss: 1.2356140613555908, G Loss: 0.8202370405197144
Epoch: 2/10, Step: 2000, D Loss: 1.2657390832901, G Loss: 0.9170143008232117
Epoch: 3/10, Step: 3000, D Loss: 1.2579013109207153, G Loss: 0.9290286302566528
Epoch: 4/10, Step: 4000, D Loss: 1.3256195783615112, G Loss: 1.0418896675109863
Epoch: 5/10, Step: 5000, D Loss: 1.2123839855194092, G Loss: 0.8865771293640137
Epoch: 6/10, Step: 6000, D Loss: 1.1807475090026855, G Loss: 0.8695520162582397
Epoch: 7/10, Step: 7000, D Loss: 1.201066017150879, G Loss: 1.041235327720642
Epoch: 8/10, Step: 8000, D Loss: 1.2208316326141357, G Loss: 1.007463812828064
Epoch: 9/10, Step: 9000, D Loss: 1.2287606000900269, G Loss: 0.8622145652770996


In [14]:
# Saving params.
torch.save(D.state_dict(), 'D.pkl')
torch.save(G.state_dict(), 'G.pkl')

In [15]:
import scipy.misc

In [35]:
# generation to image
z = cuda_var(torch.randn(1, n_noise))
fakeimg = G(z).view(28, 28)
img = fakeimg.cpu().data.numpy()
scipy.misc.toimage(img)

/home/yangyangii/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  """
